In [1]:
def addformFull(form, tag, lemm, pos):
    if form not in CorpusTaggedFull:
        CorpusTaggedFull[form] = {pos: {}}
        #if lemm not in CorpusTaggedFull[form][pos]:
        CorpusTaggedFull[form][pos][lemm] = {tag}
        #else:
        #    CorpusTaggedFull[form][pos][lemm].add(tag)
            
    else:
        if pos not in CorpusTaggedFull[form]:
            CorpusTaggedFull[form][pos] = {}
            
            #if lemm not in CorpusTagged[form][pos]:
            CorpusTaggedFull[form][pos][lemm] = set()
            CorpusTaggedFull[form][pos][lemm].add(tag)
        else:
            if lemm not in CorpusTaggedFull[form][pos]:
                CorpusTaggedFull[form][pos][lemm] = {tag}
            else:
                CorpusTaggedFull[form][pos][lemm].add(tag) 


In [2]:
#CorpusTaggedFull = {}

def getCorpusTaggedFromFile(filename):
    filepath = 'C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\data\\'
    f = open(filepath+filename, 'r', encoding='utf-8')
    w = open(filepath+'test.txt', 'w', encoding='utf-8')


    for line in f:
        try:
            form, alltags = line.strip('\n').split('\t')
            postags = alltags.split('&')
            for postag in postags:
                pos, onepostag = postag.split('$')

                lemmtags = onepostag.split('*')
                for lemmtag in lemmtags:
                    lemm, onelemmtag = lemmtag.split('+')
                    tags = onelemmtag.split('#')
                    for tag in tags:
                        addformFull(form, tag, lemm, pos)

        except:
            w.write(line)

    f.close()
    w.close()

In [3]:
CorpusTaggedFull = {}
getCorpusTaggedFromFile('adj_tagged_new.txt')

getCorpusTaggedFromFile('unchangable_tagged_new.txt')
getCorpusTaggedFromFile('noun_tagged_new.txt')
getCorpusTaggedFromFile('noun_tagged_new2.txt')
getCorpusTaggedFromFile('verbs_tagged_new.txt')



In [4]:
len(CorpusTaggedFull)

76013

In [29]:
#CorpusTaggedFull['باری']


{'ADV': {'باری': {''}},
 'AJ': {'باری': {'POS'}},
 'N': {'بار': {'SG COP2SG', 'SG YEH'}, 'باری': {'SG'}},
 'V': {'آورد': {'BE IND PRES 2SG'}, 'بارید': {'IND PRES 2SG'}}}

In [5]:

def add2bayts(bayt, semibayt, sentno, writer):
#def getprsdata():
    rowDic = {}
    #semibayt = 0

    #sentno1 = sentn-1
    #sentno2 = sentn
    #semibayt += 1
    #words = punctsplit(delimiters, b.strip())

    sent_pos = ''
    wordno = 0
    for word, taggerpos in bayt:
        #if taggerpos == 'PUNC':
        #    continue

        sent_pos = ''
        wordno += 1

        if wordno == 1:
            sent_pos = 'bos'
        elif wordno == len(bayt):
            sent_pos = 'eos'

        punct = ''   
        if sent_pos == 'eos' and semibayt == 1:
            punct = '\\t'
        elif sent_pos == 'eos' and semibayt == 2:
            punct = '\\n'


        #if semibayt == 1:
        #    sentno = sentno1
        #elif semibayt == 2:
        #    sentno = sentno2

        newtags = set()
        newlemms = set()

        if word not in CorpusTaggedFull:
            nvars = 1
            nvar = 1
            nlems = '?'
            lem = '?'
            lex = taggerpos
            gram = '?'

            rowDic['#sentno'] = sentno
            rowDic['#wordno'] = wordno
            #rowDic['#lang'] 
            #rowDic['#graph'] = graph
            rowDic['#word'] = word
            #rowDic['#indexword']
            rowDic['#nvars'] = nvars
            rowDic['#nlems'] = nlems
            rowDic['#nvar'] = nvar
            rowDic['#lem'] = lem
            #rowDic['#trans'] = trans
            #rowDic['#trans_ru']
            rowDic['#lex'] = lex
            rowDic['#gram'] = gram
            #rowDic['#flex'] = *?
            #rowDic['#punctl'] = punctl
            rowDic['#punctr'] = punct
            rowDic['#sent_pos'] = sent_pos

            writer.writerow(rowDic)
            rowDic = {}

        else:

            if taggerpos == 'V' and word.endswith('ست'):
                nvars = 0
                for pos in CorpusTaggedFull[word]:
                    for lemm in CorpusTaggedFull[word][pos]:
                        for tag in CorpusTaggedFull[word][pos][lemm]:
                            if 'COP' in tag:
                                nvars += 1
                                newtags.add((pos, lemm, tag))
                                newlemms.add(lemm)
                nlems = len(newlemms)
            

            elif taggerpos in CorpusTaggedFull[word]:
                #CorpusTaggedFull[form][pos][lemm] = {tag}

                lems = set(CorpusTaggedFull[word][taggerpos].keys())
                nlems = len(lems)
                nvars = 0
                for lemm in lems:
                    nvars += len(CorpusTaggedFull[word][taggerpos][lemm])
                    for tag in CorpusTaggedFull[word][taggerpos][lemm]:
                        newtags.add((taggerpos, lemm, tag))
                    newlemms.add(lemm)
                    
            elif taggerpos not in CorpusTaggedFull[word]: #in ('CL', 'RES', 'DET'):
                nvars = 0
                for pos in CorpusTaggedFull[word]:
                    for lemm in CorpusTaggedFull[word][pos]:
                        for tag in CorpusTaggedFull[word][pos][lemm]:
                            nvars += 1
                            newtags.add((pos, lemm, tag))
                            newlemms.add(lemm)
                nlems = len(newlemms)


            nvar = 0

            for pos, lemm, tag in newtags:
                nvar += 1
                lem = lemm
                lex = pos
                gram = tag



                rowDic['#sentno'] = sentno
                rowDic['#wordno'] = wordno
                #rowDic['#lang'] 
                #rowDic['#graph'] = graph
                rowDic['#word'] = word
                #rowDic['#indexword']
                rowDic['#nvars'] = nvars
                rowDic['#nlems'] = nlems
                rowDic['#nvar'] = nvar
                rowDic['#lem'] = lem
                #rowDic['#trans'] = trans
                #rowDic['#trans_ru']
                rowDic['#lex'] = lex
                rowDic['#gram'] = gram
                #rowDic['#flex'] = *?
                #rowDic['#punctl'] = punctl
                rowDic['#punctr'] = punct
                rowDic['#sent_pos'] = sent_pos

                writer.writerow(rowDic)
                rowDic = {}


In [6]:
import csv
teststring = ''
def toPRS(infile, outfile):
#path = 'C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\collect\\POStagged\\fa\\9\\13\\47\\'
#infile = path+'2446.txt'
#outfile = path+ '2446.prs'
    with open(infile, 'r', encoding='utf-8') as f:
        with open(outfile, 'w', encoding='utf-8') as w:

            fieldnames = ['#sentno','#wordno','#lang','#graph','#word','#indexword','#nvars','#nlems','#nvar',
                          '#lem','#trans','#trans_ru','#lex','#gram','#flex','#punctl','#punctr','#sent_pos']

            writer = csv.DictWriter(w, fieldnames=fieldnames, delimiter = '\t', restval = '', lineterminator='\n')

            sentno = 0
            #wordno = 0
            #linelist = []
            baytlist_1 = []
            baytlist_2 = []
            bnum = 1
            for line in f:



                #sentno = 1
                if line.startswith('#meta.') or line.startswith('#sentno'):
                    if line.startswith('#meta.sentences'):
                        meta, num = line.strip('\n').split('\t')
                        newnum = int(num)*2
                        w.write(meta+'\t'+str(newnum)+'\n')
                    else:
                        w.write(line)
                elif line != '\n':
                    if line.startswith('##'):
                        sentno += 1
                        add2bayts(baytlist_2, 2, sentno, writer)
                        
                        bnum = 1
                        baytlist_1 = []
                        baytlist_2 = []
                    elif line == '#\n':
                        bnum += 1
                        sentno += 1
                        
                        add2bayts(baytlist_1, 1 , sentno, writer)

                    else:
                        try:
                            word, taggerpos = line.strip('\n').split('\t')

                            if taggerpos.endswith('e'):
                                taggerpos = taggerpos[:-1]

                            if taggerpos == 'PUNC':
                                continue

                            #linelist.append((word, taggerpos))
                            elif bnum == 1:
                                baytlist_1.append((word, taggerpos))
                            elif bnum == 2:
                                baytlist_2.append((word, taggerpos))
                        except:
                            teststring += line

In [7]:
import os


path = "C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\collect\\POStagged\\fa\\"
#newpath = 'C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\collect\\prs\\fa\\'
#os.makedirs(os.path.dirname(filenamefa), exist_ok=True)

count = 0
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        oldfile = '\\'.join((dirpath, filename))
        #print('\\'.join((dirpath, filename)))
        newfile = oldfile.replace('POStagged', 'PRSnew3')
        newfile = newfile.replace('.txt', '.prs')
        
        os.makedirs(os.path.dirname(newfile), exist_ok=True)
        
        #taggedfile = open(newfile, 'w', encoding='utf-8')
        #writer = csv.DictWriter(taggedfile, fieldnames=fieldnames, delimiter = '\t', restval = '', lineterminator='\n')
        #writer.writeheader()
        
        #print(oldfile)
        #print(newfile)
        try:
            toPRS(oldfile, newfile)
        except:
            print()
        

In [21]:
century = {'4':'11', '33':'13,14', '35':'12', '15':'14', '22':'11', 
           '16':'15', '1':'14', '17':'12', '21':'14', '7':'11,12', 
           '34':'10', '14':'9,10', '29':'13', '28':'11,12', '8':'14', 
           '13':'13,14', '3':'16,17', '12':'17', '23':'14', '20':'12,13', 
           '9':'11', '19':'11', '32':'10,11', '18':'11', '2':'16', '24':'11,12', 
           '26':'11', '31':'13', '27':'11', '30':'12,13', '10': '', '11':'', '25':'', '5':'', '6':''}

In [22]:
len(century)

35

In [24]:
import os

import re

path = "C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\collect\\PRSnew3\\fa\\"
#newpath = 'C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\collect\\prs\\fa\\'
#os.makedirs(os.path.dirname(filenamefa), exist_ok=True)

count = 0
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        oldfile = '\\'.join((dirpath, filename))
        #print('\\'.join((dirpath, filename)))
        newfile = oldfile.replace('PRSnew3', 'PRSnew5')
        #newfile = newfile.replace('.txt', '.prs')
        
        os.makedirs(os.path.dirname(newfile), exist_ok=True)
        
        #taggedfile = open(newfile, 'w', encoding='utf-8')
        #writer = csv.DictWriter(taggedfile, fieldnames=fieldnames, delimiter = '\t', restval = '', lineterminator='\n')
        #writer.writeheader()
        
        #print(oldfile)
        #print(newfile)
        with open(oldfile, 'r', encoding='utf-8') as f:
            with open(newfile, 'w', encoding='utf-8') as w:
                for line in f:
                    if line.startswith('#meta.date1'):
                        
                        left, right = oldfile.split('fa\\')
                        authorid = right.split('\\')[0]
                        
                        #meta, cent = line.strip('\n').split('\t')
                        #newnum = int(num)*2
                        meta = '#meta.date1'
                        
                        w.write(meta+'\t'+str(century[authorid])+'\n')
                    else:
                        w.write(line)